In [4]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score,  classification_report

In [5]:
mobilenetv2_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi\mobilenetv2_Percobaan 4(2).h5")
xception_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi\Percobaan3_Xception.h5") 
efficientnetb0_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Model_Skripsi\Percobaan3(2)_EfficientNetB0.h5")

In [6]:
dic = {0: 'Buras', 1: 'Dangkot', 2: 'Gogos', 3: 'Kapurung', 4: 'Sokko', 5: 'Sop Konro'} 
label_to_class = {v: k for k, v in dic.items()}

In [7]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
validation_generator = datagen.flow_from_directory(
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Split_dataset\val",
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

Found 1020 images belonging to 6 classes.


In [8]:
# Get predictions from each model
mobilenetv2_preds = mobilenetv2_model.predict(validation_generator)
xception_preds = xception_model.predict(validation_generator)
efficientnetb0_preds = efficientnetb0_model.predict(validation_generator)

d:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 92s 6s/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step


In [9]:
# Average the predictions
ensemble_preds = (mobilenetv2_preds + xception_preds + efficientnetb0_preds) / 3

In [10]:
final_preds = np.argmax(ensemble_preds, axis=1)
true_labels = validation_generator.classes

In [11]:
# Calculate accuracy
accuracy = np.sum(final_preds == true_labels) / len(true_labels)

# Calculate precision, recall, and f1-score
precision = precision_score(true_labels, final_preds, average='weighted')
recall = recall_score(true_labels, final_preds, average='weighted')
f1 = f1_score(true_labels, final_preds, average='weighted')

In [12]:
# Calculate loss (categorical crossentropy)
loss = -np.sum(np.log(np.max(ensemble_preds, axis=1))) / len(true_labels)

In [13]:
print(f'Ensemble Model Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print(f'Loss: {loss:.2f}')

Ensemble Model Accuracy: 0.92
Precision: 0.92
Recall: 0.92
F1 Score: 0.92
Loss: 0.44


In [14]:
print("Classification Report: \n", classification_report(final_preds, true_labels, target_names=list(dic.values())))

Classification Report: 
               precision    recall  f1-score   support

       Buras       0.97      0.93      0.95       177
     Dangkot       0.90      0.89      0.89       172
       Gogos       0.97      0.96      0.96       172
    Kapurung       0.95      0.88      0.91       184
       Sokko       0.91      0.94      0.93       165
   Sop Konro       0.84      0.95      0.89       150

    accuracy                           0.92      1020
   macro avg       0.92      0.92      0.92      1020
weighted avg       0.93      0.92      0.92      1020

